In [92]:
import ast
import glob
import warnings
from collections import defaultdict
from datetime import date

import numpy as np
import pandas as pd
import wandb

today = date.today()
api = wandb.Api()

# # Find all csv files in the current directory
csv_files = glob.glob("*.csv")
# # Collect all the names of the csv files without the extension
csv_names = [csv_file[:-4] for csv_file in csv_files]
project_name = "graph_baselines_graph_baselines"  # rebutal_CWN_ogbg-molhiv fix_gnn_rebuttal_cell_NCI109  fix_gnn_rebuttal_cell_MUTAG fix_gnn_rebuttal_cell_PROTEINS fix_gnn_rebuttal_cell_ZINC
user = "levsap" #"telyatnikov_sap"

if project_name not in csv_names:
    runs = api.runs(f"{user}/{project_name}")

    summary_list, config_list, name_list = [], [], []
    for run in runs:
        # .summary contains the output keys/values for metrics like accuracy.
        #  We call ._json_dict to omit large files
        summary_list.append(run.summary._json_dict)

        # .config contains the hyperparameters.
        #  We remove special values that start with _.
        config_list.append(
            {k: v for k, v in run.config.items() if not k.startswith("_")}
        )

        # .name is the human-readable name of the run.
        name_list.append(run.name)

    runs_df = pd.DataFrame(
        {"summary": summary_list, "config": config_list, "name": name_list}
    )

    runs_df.to_csv(f"{project_name}.csv")
else:
    runs_df = pd.read_csv(f"{project_name}.csv", index_col=0)

    for row in runs_df.iloc:
        row["summary"] = ast.literal_eval(row["summary"])
        row["config"] = ast.literal_eval(row["config"])


for row in runs_df.iloc:
    row["summary"].update(row["config"])

lst = [i["summary"] for i in runs_df.iloc]
df = pd.DataFrame.from_dict(lst)

df_init = df.copy()

# Get average epoch run time
#df["epoch_run_time"] = df["_runtime"] / df["epoch"]

In [93]:
df["epoch_run_time"] = df["_runtime"] / df["epoch"]

In [94]:
df.shape

(7679, 104)

In [95]:
def normalize_column(df, column_to_normalize):
    # Use json_normalize to flatten the nested dictionaries into separate columns
    flattened_df = pd.json_normalize(df[column_to_normalize])
    # Rename columns to include 'nested_column' prefix
    flattened_df.columns = [
        f"{column_to_normalize}.{col}" for col in flattened_df.columns
    ]
    # Concatenate the flattened DataFrame with the original DataFrame
    result_df = pd.concat([df, flattened_df], axis=1)
    # Get new columns names
    new_columns = flattened_df.columns
    # Drop the original nested column if needed
    result_df.drop(column_to_normalize, axis=1, inplace=True)
    return result_df, new_columns


# Config columns to normalize
columns_to_normalize = ["model", "dataset", "callbacks", "paths",'optimizer' ] # 'optimizer'

# Keep track of config columns added
config_columns = []
for column in columns_to_normalize:
    df, columns = normalize_column(df, column)
    config_columns.extend(columns)

In [96]:
# Remove columns that are not needed (we shouldn't vary them or their variation is not interesting)
remove_cols = [
    #"dataset.transforms.data_manipulations.selected_fields",
    "callbacks.model_checkpoint.dirpath", "paths.output_dir", 'model.feature_encoder.selected_dimensions','callbacks.model_checkpoint.dirpath',
]
removed_columns = []
for remove_col in remove_cols:
    if remove_col in df.columns:
        df = df.drop([remove_col], axis=1)
        removed_columns.append(remove_col)

print("Removed columns:", removed_columns)

# Ensure that removed columns are not present in config_columns
config_columns = [col for col in config_columns if col not in removed_columns]

Removed columns: ['callbacks.model_checkpoint.dirpath', 'paths.output_dir']


In [97]:
[i for i in list(df.columns) if 'best' in i]

['best_epoch',
 'best_epoch/checkpoint',
 'best_epoch/train/accuracy',
 'best_epoch/train/auroc',
 'best_epoch/train/average_precision',
 'best_epoch/train/cohen_kappa',
 'best_epoch/train/f1_score',
 'best_epoch/train/jaccard',
 'best_epoch/train/loss',
 'best_epoch/train/mcc',
 'best_epoch/train/precision',
 'best_epoch/train/recall',
 'best_epoch/val/accuracy',
 'best_epoch/val/auroc',
 'best_epoch/val/average_precision',
 'best_epoch/val/cohen_kappa',
 'best_epoch/val/f1_score',
 'best_epoch/val/jaccard',
 'best_epoch/val/loss',
 'best_epoch/val/mcc',
 'best_epoch/val/precision',
 'best_epoch/val/recall',
 'best_epoch/train/mae',
 'best_epoch/train/mse',
 'best_epoch/train/r2',
 'best_epoch/val/mae',
 'best_epoch/val/mse',
 'best_epoch/val/r2',
 'callbacks.best_epoch_metrics.mode',
 'callbacks.best_epoch_metrics.monitor',
 'callbacks.best_epoch_metrics._target_']

In [98]:
# First get rid of the runs that have less than 50 epochs (minimum amount of epochs is 50)
MIN_EPOCHS = 50
df = df[df["epoch"] >= MIN_EPOCHS]

In [99]:
print(
    f"Number of rows with model.backbone._target_ = nan is {sum(df['model.backbone._target_'].isna())}"
)
# Drop na values if there are
df = df.dropna(subset=["model.backbone._target_"])
# Reset index
df = df.reset_index(drop=True)

# Drop rows that 'callbacks.early_stopping.monitor' isna
print(
    f"Number of rows with callbacks.early_stopping.monitor = nan is {sum(df['callbacks.early_stopping.monitor'].isna())}"
)

# print("Because of SCCN and CWN false runs there were 96 such runs on 13/03/24")

df = df.dropna(subset=["callbacks.early_stopping.monitor"])
# Reset index
df = df.reset_index(drop=True)


# Get correct names for the models
df["model.backbone._target_"] = df["model.backbone._target_"].apply(
    lambda x: x.split(".")[-1]
)

Number of rows with model.backbone._target_ = nan is 0
Number of rows with callbacks.early_stopping.monitor = nan is 0


In [100]:
df["model.backbone._target_"].unique()

array(['GCN', 'GIN', 'GAT'], dtype=object)

In [105]:
import pandas as pd
import numpy as np
import warnings
from collections import defaultdict
from IPython.display import display

# =============================================================================
# 1. Configuration & Constants
# =============================================================================
REQUIRED_SEEDS = {0, 3, 5, 7, 9}
SEED_COL = "dataset.split_params.data_seed"

TASK_METRICS = {
    "classification": ["accuracy", "precision", "recall", "f1_score", "cohen_kappa", "mcc", "jaccard"],
    "regression": ["mse", "mae", "r2"]
}

def get_performance_cols(subset, task_type):
    found_cols = []
    keywords = TASK_METRICS.get(task_type, [])
    
    for col in subset.columns:
        if col.startswith("test/"):
            metric_name = col.split("/")[-1]
            if metric_name in keywords:
                found_cols.append(col)
    
    try:
        opt_metric_path = subset["callbacks.early_stopping.monitor"].iloc[0]
        opt_metric_name = opt_metric_path.split("/")[-1]
        test_opt_metric = f"test/{opt_metric_name}"
        if test_opt_metric not in found_cols and test_opt_metric in subset.columns:
            found_cols.append(test_opt_metric)
    except:
        pass 
    return list(set(found_cols))

def get_metric_name(subset):
    metric_path = subset["callbacks.early_stopping.monitor"].unique()
    if len(metric_path) != 1:
        raise ValueError(f"CRITICAL ERROR: Multiple monitor metrics found: {metric_path}")
    return metric_path[0].split("/")[-1]

# =============================================================================
# 2. Core Processing Logic
# =============================================================================
def process_dataset_model(df, dataset, model, config_columns):
    subset = df[
        (df["dataset.loader.parameters.data_name"] == dataset)
        & (df["model.backbone._target_"] == model)
    ].copy()

    if subset.empty:
        return None

    warnings.filterwarnings("ignore")
    subset["Model"] = model
    warnings.filterwarnings("default")

    # STRICT Mode Check
    modes = subset["callbacks.early_stopping.mode"].unique()
    if len(modes) == 0:
        raise ValueError(f"CRITICAL ERROR: No early_stopping mode found for {model} on {dataset}.")
    if len(modes) > 1:
        raise ValueError(f"CRITICAL ERROR: Ambiguous early_stopping modes {modes} for {model} on {dataset}.")
    
    mode = modes[0]
    if mode == "max":
        task_type = "classification"
        ascending = False
    elif mode == "min":
        task_type = "regression"
        ascending = True
    else:
        raise ValueError(f"CRITICAL ERROR: Invalid early_stopping mode '{mode}'")

    performance_cols = get_performance_cols(subset, task_type)

    subset[performance_cols] = subset[performance_cols].apply(pd.to_numeric, errors='coerce')
    nan_mask = subset[performance_cols].isna().any(axis=1)
    subset = subset[~nan_mask].reset_index(drop=True)
    
    if subset.empty:
        return None

    # Determine HPs
    unique_colums_values = {}
    check_cols = list(config_columns)
    if SEED_COL not in check_cols and SEED_COL in subset.columns:
        check_cols.append(SEED_COL)

    for col in check_cols:
        if col in subset.columns:
            try:
                unique_vals = subset[col].apply(lambda x: tuple(x) if isinstance(x, list) else x).unique()
                if len(unique_vals) > 1:
                    sorted_vals = sorted(list(unique_vals), key=lambda x: str(x))
                    unique_colums_values[col] = sorted_vals
            except Exception:
                pass

    print("---------------------------------------------------------------")
    print(f"UNIQUE VALUES FOR MODEL: {model}; DATASET: {dataset} ({task_type.upper()}):")
    print("---------------------------------------------------------------")
    for col, unique in unique_colums_values.items():
        print(f"{col}: {unique}")
    print("---------------------------END---------------------------------")

    if SEED_COL in unique_colums_values:
        unique_colums_values.pop(SEED_COL)

    aggregation_columns = ["Model"] + list(unique_colums_values.keys())

    # Filter Seeds
    def has_required_seeds(group):
        current_seeds = set(group[SEED_COL].unique())
        if current_seeds != REQUIRED_SEEDS: return False
        if len(group) != len(REQUIRED_SEEDS): return False
        return True
    
    subset_filtered = subset.groupby(aggregation_columns).filter(has_required_seeds)
    if subset_filtered.empty:
        return None
        
    # Safety Check for Hidden Variations
    potential_hps = [
        c for c in subset.columns 
        if not c.startswith("test/") and not c.startswith("val/") and not c.startswith("train/")
        and c not in ["mean", "std", "Model", SEED_COL] and "time" not in c.lower() and "dir" not in c.lower()
    ]
    for col in potential_hps:
        if col in aggregation_columns: continue
        try:
            if len(subset_filtered[col].unique()) > 1:
                print(f"[WARNING] Hidden variation detected in '{col}'! You might be merging distinct runs.")
        except: pass

    num_unique_configs = len(subset_filtered.groupby(aggregation_columns))
    valid_grid = {}
    for col in aggregation_columns:
        if col == "Model": continue
        u_vals = subset_filtered[col].apply(lambda x: tuple(x) if isinstance(x, list) else x).unique()
        valid_grid[col] = sorted(list(u_vals), key=lambda x: str(x))

    # Aggregation
    agg_dict = {col: ["mean", "std"] for col in performance_cols}
    aggregated = subset_filtered.groupby(aggregation_columns).agg(agg_dict).reset_index()

    # Sort Results
    optimized_metric_name = get_metric_name(subset)
    sort_col = f"test/{optimized_metric_name}"
    
    if (sort_col, "mean") not in aggregated.columns:
        raise ValueError(f"Metric '{sort_col}' not found in results.")

    leaderboard = aggregated.sort_values(
        by=(sort_col, "mean"), 
        ascending=ascending
    ).copy()

    # -------------------------------------------------------------------------
    # CORRECT SCALING LOGIC (Fixes the 6400 issue)
    # -------------------------------------------------------------------------
    # Identify strictly the metric columns (tuples starting with test/)
    # and NOT the hyperparameter columns.
    
    cols_to_scale = []
    
    for col in leaderboard.columns:
        # Check if column is a tuple and belongs to performance_cols
        if isinstance(col, tuple) and col[0] in performance_cols:
            cols_to_scale.append(col)
            
    if mode == "max":
        if cols_to_scale:
            leaderboard.loc[:, cols_to_scale] = (leaderboard.loc[:, cols_to_scale] * 100).round(2)
    else:
        if cols_to_scale:
            leaderboard.loc[:, cols_to_scale] = leaderboard.loc[:, cols_to_scale].round(4)
    # -------------------------------------------------------------------------

    best_row_df = leaderboard.head(1)

    result_data = {
        "dataset": dataset,
        "model": model,
        "task_type": task_type,
        "optimized_metric": optimized_metric_name,
        "num_configs": num_unique_configs,
        "hyperparameter_grid": valid_grid,
        "best_config": best_row_df,
        "full_leaderboard": leaderboard,
        "aggregation_columns": aggregation_columns
    }

    for metric_col in performance_cols:
        mean_val = best_row_df[(metric_col, "mean")].values[0]
        std_val = best_row_df[(metric_col, "std")].values[0]
        simple_name = metric_col.split("/")[-1]
        result_data[simple_name] = f"{mean_val:.2f} ± {std_val:.2f}"

    return result_data

# =============================================================================
# 3. Execution
# =============================================================================
unique_models = df["model.backbone._target_"].unique()
unique_datasets = df["dataset.loader.parameters.data_name"].unique()

all_results = []
print("Processing results...")

for dataset in unique_datasets:
    for model in unique_models:
        try:
            res = process_dataset_model(df, dataset, model, config_columns)
            if res:
                all_results.append(res)
        except ValueError as e:
            print(f"\n!!! EXECUTION STOPPED !!!\n{e}")
            raise e 

print(f"Done. Processed {len(all_results)} pairs.")

# =============================================================================
# 4. Display Functions
# =============================================================================

def display_performance_tables(results_list):
    if not results_list: return
    df_results = pd.DataFrame(results_list)

    for task in ["classification", "regression"]:
        task_df = df_results[df_results["task_type"] == task]
        if task_df.empty: continue
            
        print(f"\n{'='*40}\n  PERFORMANCE: {task.upper()}\n{'='*40}")
        ignore_cols = ["dataset", "model", "task_type", "optimized_metric", "best_config", "num_configs", "hyperparameter_grid", "full_leaderboard", "aggregation_columns"]
        metric_cols = [c for c in task_df.columns if c not in ignore_cols]
        
        for metric in metric_cols:
            pivot = task_df.pivot_table(index="model", columns="dataset", values=metric, aggfunc="first").fillna("-")
            print(f"\nMetric: {metric.upper()}")
            display(pivot)

def display_experiment_stats(results_list):
    if not results_list: return
    df_results = pd.DataFrame(results_list)
    count_pivot = df_results.pivot_table(
        index="model", columns="dataset", values="num_configs", aggfunc="first"
    ).fillna(0).astype(int)
    print(f"\n{'='*40}\n  COMPLETED HP CONFIGURATIONS (Count)\n{'='*40}")
    display(count_pivot)

def display_hp_impact(results_list):
    if not results_list: return
    
    print(f"\n{'='*40}\n  HYPERPARAMETER IMPACT ANALYSIS\n{'='*40}")
    print("Listing all configurations sorted by performance.")
    
    for entry in results_list:
        d = entry['dataset']
        m = entry['model']
        leaderboard = entry['full_leaderboard'].copy() 
        opt_metric = entry['optimized_metric']
        
        # 1. ROBUST FLATTENING: Convert MultiIndex columns to Strings
        if isinstance(leaderboard.columns, pd.MultiIndex):
            new_cols = []
            for col in leaderboard.columns:
                c0 = str(col[0])
                c1 = str(col[1])
                if c1 == "":
                    new_cols.append(c0)
                else:
                    new_cols.append(f"{c0}|{c1}") 
            leaderboard.columns = new_cols

            metric_base = f"test/{opt_metric}"
            target_mean = f"{metric_base}|mean"
            target_std = f"{metric_base}|std"
        else:
            target_mean = f"test/{opt_metric}" 
            target_std = f"test/{opt_metric}_std"

        # 2. Select Columns
        hp_cols = [h for h in entry['aggregation_columns'] if h != "Model"]
        selected_cols = []
        rename_map = {}
        
        for hp in hp_cols:
            if hp in leaderboard.columns:
                selected_cols.append(hp)
        
        if target_mean in leaderboard.columns:
            selected_cols.append(target_mean)
            rename_map[target_mean] = f"{opt_metric} (Mean)"
            
        if target_std in leaderboard.columns:
            selected_cols.append(target_std)
            rename_map[target_std] = f"{opt_metric} (Std)"
            
        if not selected_cols:
             print(f"\n>>> {m} on {d}: Columns not found.")
             continue
             
        # 3. Create Final View
        final_view = leaderboard[selected_cols].rename(columns=rename_map)
        
        print(f"\n>>> {m} on {d} (Sorted by {opt_metric})")
        display(final_view)

# =============================================================================
# 5. Run All Displays
# =============================================================================
display_experiment_stats(all_results)
display_performance_tables(all_results)
display_hp_impact(all_results)

Processing results...
---------------------------------------------------------------
UNIQUE VALUES FOR MODEL: GCN; DATASET: hm-categories (CLASSIFICATION):
---------------------------------------------------------------
model.readout.hidden_dim: [16, 32, 64]
model.backbone.num_layers: [1, 2, 4, 8]
model.backbone.in_channels: [16, 32, 64]
model.backbone.hidden_channels: [16, 32, 64]
model.feature_encoder.out_channels: [16, 32, 64]
model.backbone_wrapper.out_channels: [16, 32, 64]
dataset.split_params.data_seed: [0, 3, 5, 7, 9]
optimizer.parameters.lr: [0.001, 0.01]
---------------------------END---------------------------------
[WARNING] Hidden variation detected in '_step'! You might be merging distinct runs.
[WARNING] Hidden variation detected in 'best_epoch'! You might be merging distinct runs.
[WARNING] Hidden variation detected in 'best_epoch/checkpoint'! You might be merging distinct runs.
[WARNING] Hidden variation detected in 'best_epoch/train/accuracy'! You might be merging di

dataset,Cora,PubMed,amazon_ratings,artnet-exp,artnet-views,avazu-ctr,citeseer,city-reviews,city-roads-L,city-roads-M,hm-categories,pokec-regions,roman_empire,tolokers-2,twitch-views
model,,,,,,,,,,,,,,,
GAT,30,32,32,32,30,6,29,30,32,31,6,2,32,32,10
GCN,31,32,32,31,32,24,31,32,32,31,24,14,32,32,32
GIN,32,30,32,29,31,24,29,32,32,32,32,22,32,32,29



  PERFORMANCE: CLASSIFICATION

Metric: RECALL


dataset,Cora,PubMed,amazon_ratings,artnet-exp,citeseer,city-reviews,hm-categories,pokec-regions,roman_empire,tolokers-2
model,,,,,,,,,,
GAT,85.33 ± 0.74,89.02 ± 0.40,42.46 ± 0.44,60.84 ± 0.68,69.72 ± 1.36,81.58 ± 0.25,30.86 ± 0.67,1.15 ± 0.01,72.16 ± 0.81,65.31 ± 0.31
GCN,84.83 ± 3.59,89.19 ± 0.36,42.83 ± 0.88,62.88 ± 0.76,70.30 ± 1.70,81.68 ± 0.25,33.23 ± 0.42,2.49 ± 0.12,70.19 ± 0.87,67.05 ± 1.18
GIN,85.27 ± 1.08,88.94 ± 0.54,39.00 ± 1.74,60.90 ± 0.93,69.90 ± 1.38,80.64 ± 0.18,40.75 ± 0.50,3.26 ± 0.05,71.11 ± 1.06,60.34 ± 0.61



Metric: JACCARD


...

,model.readout.hidden_dim,model.backbone.num_layers,model.backbone.in_channels,model.backbone.hidden_channels,model.feature_encoder.out_channels,model.backbone_wrapper.out_channels,optimizer.parameters.lr,accuracy (Mean),accuracy (Std)
17,64,1,64,64,64,64,0.010,76.78,0.64
25,128,1,128,128,128,128,0.010,76.37,0.47
9,32,1,32,32,32,32,0.010,76.03,0.57
27,128,2,128,128,128,128,0.010,75.47,0.39
19,64,2,64,64,64,64,0.010,75.46,0.69
24,128,1,128,128,128,128,0.001,74.73,0.51
11,32,2,32,32,32,32,0.010,74.12,0.25
26,128,2,128,128,128,128,0.001,73.68,0.50
1,16,1,16,16,16,16,0.010,73.55,0.41
21,64,4,64,64,64,64,0.010,73.45,0.38



>>> GIN on roman_empire (Sorted by accuracy)


,model.readout.hidden_dim,model.backbone.num_layers,model.backbone.in_channels,model.backbone.hidden_channels,model.feature_encoder.out_channels,model.backbone_wrapper.out_channels,optimizer.parameters.lr,accuracy (Mean),accuracy (Std)
17,64,1,64,64,64,64,0.010,77.79,0.82
25,128,1,128,128,128,128,0.010,77.39,0.15
9,32,1,32,32,32,32,0.010,77.32,0.58
24,128,1,128,128,128,128,0.001,77.26,0.37
26,128,2,128,128,128,128,0.001,75.53,0.19
19,64,2,64,64,64,64,0.010,75.40,0.76
1,16,1,16,16,16,16,0.010,74.90,0.39
11,32,2,32,32,32,32,0.010,74.77,0.42
16,64,1,64,64,64,64,0.001,74.35,0.59
28,128,4,128,128,128,128,0.001,73.63,0.30



>>> GAT on roman_empire (Sorted by accuracy)


,model.readout.hidden_dim,model.backbone.num_layers,model.backbone.in_channels,model.backbone.hidden_channels,model.feature_encoder.out_channels,model.backbone_wrapper.out_channels,optimizer.parameters.lr,accuracy (Mean),accuracy (Std)
27,128,2,128,128,128,128,0.010,77.84,0.45
17,64,1,64,64,64,64,0.010,77.54,0.27
25,128,1,128,128,128,128,0.010,77.45,0.55
19,64,2,64,64,64,64,0.010,77.32,0.50
9,32,1,32,32,32,32,0.010,76.68,0.61
11,32,2,32,32,32,32,0.010,75.54,0.68
24,128,1,128,128,128,128,0.001,75.37,0.22
26,128,2,128,128,128,128,0.001,74.73,0.62
1,16,1,16,16,16,16,0.010,74.35,0.54
21,64,4,64,64,64,64,0.010,74.23,0.34


In [70]:
df['optimizer.parameters.lr']

KeyError: 'optimizer.parameters.lr'

In [55]:
# Convert nested dictionary to DataFrame
nested_dict_perf = dict(collected_results)
performance_df = pd.DataFrame.from_dict(
    {
        (i, j): nested_dict_perf[i][j]
        for i in nested_dict_perf
        for j in nested_dict_perf[i].keys()
    },
    orient="index",
)

# Format as "Mean ± Std"
performance_df = performance_df.round(2)
performance_df["performance"] = performance_df.apply(
    lambda x: f"{x['mean']} ± {x['std']}", axis=1
)

# Clean up columns
performance_df = performance_df.drop(["mean", "std"], axis=1)
performance_df = performance_df.reset_index()
performance_df.columns = ["Dataset", "Model", "Performance"]

# Create Pivot Table
performance_pivot = performance_df.pivot_table(
    index="Model", columns="Dataset", values="Performance", aggfunc="first"
)

print("Performance Table:")
display(performance_pivot)

Performance Table:


Dataset,Cora,PubMed,amazon_ratings,artnet-exp,artnet-views,avazu-ctr,citeseer,city-reviews,city-roads-L,city-roads-M,hm-categories,pokec-regions,roman_empire,tolokers-2,twitch-views
Model,,,,,,,,,,,,,,,
GAT,86.25 ± 1.1,89.26 ± 0.28,49.89 ± 0.99,90.82 ± 0.28,2.04 ± 0.48,0.02 ± 0.0,73.58 ± 1.54,93.8 ± 0.11,11.72 ± 1.67,10.66 ± 2.09,34.2 ± 8.28,4.66 ± 0.91,76.41 ± 1.16,81.07 ± 0.8,23.92 ± 16.53
GCN,85.83 ± 0.82,89.38 ± 0.26,49.79 ± 1.03,91.08 ± 0.26,1.85 ± 0.22,0.02 ± 0.0,73.24 ± 1.48,93.95 ± 0.1,11.85 ± 1.46,10.71 ± 1.81,35.58 ± 5.81,6.57 ± 2.39,75.55 ± 0.98,81.84 ± 0.61,5.4 ± 0.66
GIN,85.66 ± 1.13,88.99 ± 0.48,47.47 ± 0.89,90.74 ± 0.31,1.53 ± 0.14,0.03 ± 0.0,73.22 ± 1.4,93.56 ± 0.09,11.76 ± 1.25,10.78 ± 1.62,48.64 ± 2.49,9.99 ± 0.38,77.33 ± 0.28,80.38 ± 1.06,6.29 ± 0.3


In [56]:
# ---------------------------------------------------------
# 1. Create a map of {Dataset_Name -> Metric_Name}
# ---------------------------------------------------------
dataset_metric_map = {}

# We look into the 'collected_non_aggregated_results' dictionary to find the metric name used
for dataset in performance_pivot.columns:
    if dataset in collected_non_aggregated_results:
        # Grab any available model for this dataset to inspect the metric
        # (The metric is the same for all models on a given dataset)
        first_model_key = next(iter(collected_non_aggregated_results[dataset]))
        
        # Extract the performance columns list
        metrics_list = collected_non_aggregated_results[dataset][first_model_key]["performance_cols"]
        
        # Join them into a string (e.g., "test/mse")
        dataset_metric_map[dataset] = ", ".join(metrics_list)
    else:
        dataset_metric_map[dataset] = "Unknown"

# ---------------------------------------------------------
# 2. Create the Footer Row
# ---------------------------------------------------------
# Create a single-row DataFrame matching the columns of your pivot table
metric_row = pd.DataFrame(
    [[dataset_metric_map[col] for col in performance_pivot.columns]], 
    columns=performance_pivot.columns, 
    index=["METRIC"]
)

# ---------------------------------------------------------
# 3. Concatenate and Display
# ---------------------------------------------------------
final_performance_table = pd.concat([performance_pivot, metric_row])

print("Final Performance Table with Metrics:")
display(final_performance_table)

Final Performance Table with Metrics:


Dataset,Cora,PubMed,amazon_ratings,artnet-exp,artnet-views,avazu-ctr,citeseer,city-reviews,city-roads-L,city-roads-M,hm-categories,pokec-regions,roman_empire,tolokers-2,twitch-views
GAT,86.25 ± 1.1,89.26 ± 0.28,49.89 ± 0.99,90.82 ± 0.28,2.04 ± 0.48,0.02 ± 0.0,73.58 ± 1.54,93.8 ± 0.11,11.72 ± 1.67,10.66 ± 2.09,34.2 ± 8.28,4.66 ± 0.91,76.41 ± 1.16,81.07 ± 0.8,23.92 ± 16.53
GCN,85.83 ± 0.82,89.38 ± 0.26,49.79 ± 1.03,91.08 ± 0.26,1.85 ± 0.22,0.02 ± 0.0,73.24 ± 1.48,93.95 ± 0.1,11.85 ± 1.46,10.71 ± 1.81,35.58 ± 5.81,6.57 ± 2.39,75.55 ± 0.98,81.84 ± 0.61,5.4 ± 0.66
GIN,85.66 ± 1.13,88.99 ± 0.48,47.47 ± 0.89,90.74 ± 0.31,1.53 ± 0.14,0.03 ± 0.0,73.22 ± 1.4,93.56 ± 0.09,11.76 ± 1.25,10.78 ± 1.62,48.64 ± 2.49,9.99 ± 0.38,77.33 ± 0.28,80.38 ± 1.06,6.29 ± 0.3
METRIC,test/accuracy,test/accuracy,test/accuracy,test/accuracy,test/mse,test/mse,test/accuracy,test/accuracy,test/mse,test/mse,test/accuracy,test/accuracy,test/accuracy,test/accuracy,test/mse


In [63]:
# ---------------------------------------------------------
# Configuration
# ---------------------------------------------------------
PROJECT_NAME = "TopoBench_Reproduction"  # <--- Change this to your desired project name
SEEDS_STR = "0,3,5,7,9"

reproduction_commands = []

for dataset in collected_aggregated_results:
    for model in collected_aggregated_results[dataset]:
        
        # 1. Get the Best Run's Hyperparameters (Varying Params)
        # -----------------------------------------------------
        best_run_row = collected_aggregated_results[dataset][model].iloc[0]
        
        # 2. Get the Full Context (Static Params)
        # -----------------------------------------------------
        full_subset = collected_non_aggregated_results[dataset][model]["df"]
        
        # Initialize command parts
        cmd_parts = [
            "python -m topobench",
            f"model={model}",
            f"dataset={dataset}", 
        ]
        
        # 3. Add The Optimized Hyperparameters
        # ------------------------------------
        for key, val in best_run_row.items():
            if key == "Model": continue
            if isinstance(key, tuple): continue 
            if key in ["mean", "std"]: continue
            if "unnamed" in str(key).lower(): continue

            if isinstance(val, (list, tuple)):
                val_str = "[" + ",".join(map(str, val)) + "]"
                cmd_parts.append(f"{key}={val_str}")
            else:
                cmd_parts.append(f"{key}={val}")

        # 4. Add Important Static Configurations
        # --------------------------------------
        relevant_prefixes = ("trainer.", "optimizer.", "dataset.", "model.", "callbacks.")
        existing_keys = set([c.split('=')[0] for c in cmd_parts])

        for col in full_subset.columns:
            if not col.startswith(relevant_prefixes): continue
            if col in existing_keys: continue
            
            # Skip specific columns we want to override or ignore
            if "seed" in col: continue   
            if "cwd" in col or "dir" in col: continue 
            if "wandb" in col: continue  # Skip original wandb params (we set project manually below)
            if "monitor" in col or "mode" in col: continue 
            if "_target_" in col: continue 

            unique_vals = full_subset[col].unique()
            if len(unique_vals) == 1:
                val = unique_vals[0]
                
                if pd.isna(val) or val == "": continue
                
                if isinstance(val, (list, tuple)):
                     val_str = "[" + ",".join(map(str, val)) + "]"
                     cmd_parts.append(f"{col}={val_str}")
                else:
                     cmd_parts.append(f"{col}={val}")

        # 5. Finalize with Seeds, Project Name, and Multirun
        # --------------------------------------------------
        cmd_parts.append(f"dataset.split_params.data_seed={SEEDS_STR}")
        cmd_parts.append(f"logger.wandb.project={PROJECT_NAME}")  # <--- NEW PROJECT NAME ADDED HERE
        cmd_parts.append("--multirun")

        # 6. Store
        reproduction_commands.append({
            "Dataset": dataset,
            "Model": model,
            "Command": " ".join(cmd_parts)
        })

# See one example 
print(reproduction_commands[0]["Command"])

python -m topobench model=GCN dataset=hm-categories dataset.loader.parameters.data_name=hm-categories dataset.loader.parameters.data_type=graphland dataset.loader.parameters.data_domain=graph dataset.loader.parameters.drop_missing_y=False dataset.loader.parameters.impute_missing_x.copy=True dataset.loader.parameters.impute_missing_x.strategy=most_frequent dataset.loader.parameters.impute_missing_x.add_indicator=False dataset.parameters.task=classification dataset.parameters.loss_type=cross_entropy dataset.parameters.task_level=node dataset.parameters.num_classes=21 dataset.parameters.num_features=35 dataset.split_params.k=10 dataset.split_params.split_type=random dataset.split_params.train_prop=0.5 dataset.split_params.learning_setting=transductive dataset.dataloader_params.batch_size=1 dataset.dataloader_params.pin_memory=False dataset.dataloader_params.num_workers=0 callbacks.early_stopping.strict=True callbacks.early_stopping.verbose=True callbacks.early_stopping.patience=10 callbac

{'Dataset': 'hm-categories',
 'Model': 'GCN',
 'Command': 'python -m topobench model=GCN dataset=hm-categories dataset.loader.parameters.data_name=hm-categories dataset.loader.parameters.data_type=graphland dataset.loader.parameters.data_domain=graph dataset.loader.parameters.drop_missing_y=False dataset.loader.parameters.impute_missing_x.copy=True dataset.loader.parameters.impute_missing_x.strategy=most_frequent dataset.loader.parameters.impute_missing_x.add_indicator=False dataset.parameters.task=classification dataset.parameters.loss_type=cross_entropy dataset.parameters.task_level=node dataset.parameters.num_classes=21 dataset.parameters.num_features=35 dataset.split_params.k=10 dataset.split_params.split_type=random dataset.split_params.train_prop=0.5 dataset.split_params.learning_setting=transductive dataset.dataloader_params.batch_size=1 dataset.dataloader_params.pin_memory=False dataset.dataloader_params.num_workers=0 callbacks.early_stopping.strict=True callbacks.early_stoppin

### Runtime per epoch

In [44]:
collected_results_time
# Convert nested dictionary to DataFrame
nested_dict = dict(collected_results_time)
result_dict = pd.DataFrame.from_dict(
    {
        (i, j): nested_dict[i][j]
        for i in nested_dict
        for j in nested_dict[i].keys()
    },
    orient="index",
)


result_dict = result_dict.round(2)
result_dict["performance"] = result_dict.apply(
    lambda x: f"{x['mean']} ± {x['std']}", axis=1
)
result_dict = result_dict.drop(["mean", "std"], axis=1)

# Reset multiindex
result_dict = result_dict.reset_index()
# rename columns
result_dict.columns = ["Dataset", "Model", "Average Time per Epoch"]

In [45]:
result_dict.pivot_table(
    index="Model", columns="Dataset", values="Average Time per Epoch", aggfunc="first"
)

Dataset,Cora,PubMed,amazon_ratings,artnet-exp,artnet-views,avazu-ctr,citeseer,city-reviews,city-roads-L,city-roads-M,hm-categories,pokec-regions,roman_empire,tolokers-2,twitch-views
Model,,,,,,,,,,,,,,,
GAT,0.19 ± 0.01,0.24 ± 0.02,0.04 ± 0.02,0.06 ± 0.04,0.05 ± 0.04,0.52 ± 0.05,0.25 ± 0.01,0.16 ± 0.09,0.05 ± 0.03,0.03 ± 0.02,0.42 ± 0.04,2.15 ± 0.05,0.03 ± 0.01,0.07 ± 0.06,0.33 ± 0.07
GCN,0.18 ± 0.01,0.23 ± 0.01,0.03 ± 0.01,0.03 ± 0.01,0.03 ± 0.01,0.56 ± 0.18,0.25 ± 0.01,0.1 ± 0.04,0.03 ± 0.01,0.02 ± 0.01,0.51 ± 0.17,11.82 ± 7.48,0.03 ± 0.0,0.03 ± 0.02,0.39 ± 0.23
GIN,0.18 ± 0.01,0.23 ± 0.01,0.02 ± 0.0,0.03 ± 0.01,0.03 ± 0.01,0.52 ± 0.16,0.25 ± 0.01,0.09 ± 0.04,0.03 ± 0.01,0.02 ± 0.01,0.52 ± 0.3,2.26 ± 0.16,0.03 ± 0.0,0.03 ± 0.01,0.31 ± 0.14


### Runtime

In [47]:
collected_results_time_run
# Convert nested dictionary to DataFrame
nested_dict = dict(collected_results_time_run)
result_dict = pd.DataFrame.from_dict(
    {
        (i, j): nested_dict[i][j]
        for i in nested_dict
        for j in nested_dict[i].keys()
    },
    orient="index",
)


result_dict = result_dict.round(2)
result_dict["performance"] = result_dict.apply(
    lambda x: f"{x['mean']} ± {x['std']}", axis=1
)
result_dict = result_dict.drop(["mean", "std"], axis=1)

# Reset multiindex
result_dict = result_dict.reset_index()
# rename columns
result_dict.columns = ["Dataset", "Model", "Average Training Time"]


In [48]:
result_dict.pivot_table(
    index="Model", columns="Dataset", values="Average Training Time", aggfunc="first"
)

Dataset,Cora,PubMed,amazon_ratings,artnet-exp,artnet-views,avazu-ctr,citeseer,city-reviews,city-roads-L,city-roads-M,hm-categories,pokec-regions,roman_empire,tolokers-2,twitch-views
Model,,,,,,,,,,,,,,,
GAT,32.97 ± 17.08,42.8 ± 18.93,14.31 ± 7.73,12.48 ± 7.04,20.59 ± 19.19,99.65 ± 40.46,26.94 ± 12.52,68.29 ± 42.31,33.99 ± 21.12,18.57 ± 9.99,205.67 ± 42.4,1036.01 ± 168.84,13.36 ± 5.01,16.68 ± 13.98,170.26 ± 67.39
GCN,31.84 ± 14.85,38.52 ± 18.1,10.63 ± 3.34,12.31 ± 6.14,13.12 ± 4.77,141.18 ± 64.78,25.86 ± 10.97,40.85 ± 17.66,14.81 ± 6.96,9.59 ± 3.67,293.66 ± 124.7,17228.85 ± 12894.43,11.14 ± 3.43,10.68 ± 7.64,173.02 ± 95.12
GIN,25.3 ± 10.54,49.25 ± 25.52,7.72 ± 3.09,5.71 ± 2.94,8.83 ± 4.08,78.36 ± 49.69,26.95 ± 11.26,25.99 ± 17.45,11.49 ± 6.96,7.7 ± 3.95,112.08 ± 77.15,774.89 ± 376.36,11.45 ± 3.46,4.74 ± 2.02,112.97 ± 72.32
